In [1]:
import requests
import json
import sys
import shutil
import os
from pdfrw import PdfReader, PdfWriter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from io import StringIO
import re
import pandas as pd
from tqdm import tqdm
import fitz
from datetime import datetime
from pretty_html_table import build_table
import pdfkit

In [2]:
from pdfrw import PdfReader, PdfWriter
import os
import shutil
from utils import \
    check_status, \
    create_filedir, \
    check_input_file, \
    pdf_merger, \
    read_config, \
    convert_pdf_to_string

In [3]:
check_status()
create_filedir()

In [ ]:
all_pdf = check_input_file("input")
config = read_config()
# merge pdf and remove blank pages
print("Merging all the PDF Files")
pdf_merger(all_pdf)
print("Merge Completed")

In [ ]:
print("Converting PDF to Text")
all_page = convert_pdf_to_string(os.path.join("temp", "output.pdf"))
print("Conversion Completed")

In [19]:
def sku_extract(page):
    page = re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]", "", page)
    page = page.split("\n")
    all_pipe = [x for x in page if "|" in x]
    try:
        skus = [x for x in all_pipe if x[0].isnumeric()]
    except:
        return 0
    sku = skus[0].split(" ", 1)
    return sku[1], len(skus) > 1


In [20]:
def quantity_extract(page):
    page = re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]", "", page)
    page = page.split("\n")
    qty_idx = [x for x in range(len(page)) if "QTY" in page[x]][0]
    qtys = []
    while True:
        qty_idx += 1
        if len(page[qty_idx]) > 5:
            break
        if len(page[qty_idx]) == 0:
            continue
        if page[qty_idx].isnumeric():
            qtys.append(int(page[qty_idx]))
    return sum(qtys), len(qtys) > 1

In [21]:
def courier_extract(page):
    page = re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]", "", page)
    page = page.split("\n")
    try:
        return page[2].strip()
    except:
        return ""


In [27]:
def soldBy_extract(page):
    page = re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]", "", page)
    page = page.split("\n")
    try:
        soldby_idx = [x for x in range(len(page)) if "Sold By:" in page[x]][0]
        return page[soldby_idx].replace("Sold By:", "").strip().split(",", 1)[0]
    except:
        return ""


In [32]:
pdf_path = "input/Flipkart-Labels-19-Dec-2024-10-52.pdf"
now = datetime.now()
formatted_datetime = now.strftime("%d-%m-%y %I:%M %p")
main = fitz.open(pdf_path)
invoice = fitz.open(pdf_path)
result = fitz.open()
page = main[0]
invoice_page = invoice[0]
text_instances = page.search_for("Order Id:")[0]
page_crop_rect = fitz.Rect(185, 15, page.rect.width-185, text_instances.y0 - 10)
page.set_cropbox(page_crop_rect)
p = fitz.Point(12, 10)
page.insert_text(p, formatted_datetime, fontsize=11, rotate=0)
text_instances_tax_invoice = invoice_page.search_for("Order Id:")
tax_invoice_pos = text_instances_tax_invoice[0].y1
text_instances_online_payment = invoice_page.search_for("Authorized Signature")
online_payment_pos = text_instances_online_payment[0].y0
page_crop_rect = fitz.Rect(
    0,
    tax_invoice_pos - 18,
    invoice_page.rect.width,
    online_payment_pos + 20,
)
invoice_page.set_cropbox(page_crop_rect)
result.insert_pdf(invoice, from_page=0, to_page=0, start_at=0)
result.insert_pdf(main, from_page=0, to_page=0, start_at=0)
result.save("output/result.pdf")

In [2]:
from utilsv2 import pdf_cropper, read_config
pdf_cropper("temp/output_multi.pdf", read_config())

100%|██████████| 1/1 [00:00<00:00, 27.23it/s]


In [9]:
from utilsv2 import read_config
config = read_config()